# DTI Tractography

Tractography is a 3D modeling technique used to visualize the white matter tracts within the brain. It leverages data from diffusion-weighted MRI to map out the white matter pathways, providing insights into the brain's connectivity and structural organization. 

In this tutorial we will focus on tractography based on the DTI model. The movement of water molecules tends to be more restricted along the direction of nerve fibers in white matter, and the DTI model can be used to identify the primary direction of diffusion.

## Key Concepts in DTI Tractography

1. Diffusion Tensor Imaging (DTI):

    * Diffusion Weighted Imaging (DWI): DWI captures the diffusion of water molecules in different directions. In white matter, water diffusion is anisotropic (direction-dependent), which DTI exploits to model fiber orientation.

    * Tensor Calculation: A tensor is computed for each voxel, describing the direction and magnitude of diffusion. The primary eigenvector of the tensor indicates the dominant direction of water diffusion, aligning with the direction of nerve fibers.

2. Fiber Tracking:
    
    * Seed Points: Tractography algorithms start from seed points, typically placed in regions of interest within the brain. 

    * Tracking Algorithm: A method for propagating the streamlines. There are two main types of tracking algorithms:

        * Deterministic Tractography: Follows the principal diffusion direction voxel by voxel, creating a single streamline per seed point.
        
        * Probabilistic Tractography: Accounts for uncertainty in fiber direction, generating multiple potential pathways from each seed point.

    * Stopping criteria: a set of criteria to determine when streamline propagation should terminate. 


##  DTI Deterministic Tractography

This tutorial will show how to create a tractography reconstruction from a diffusion MRI dataset (also know as tractogram), using the DTI model and deterministic tractography. and it will include examples from https://workshop.dipy.org/documentation/1.7.0/examples_built/. This procedure is called deterministic because if you repeat the exact steps below you will always get exactly the same set of streamlines. In the next tutorial we will discuss how this differs from probabilistic tractography. 

Firstly, we will load the pre-processed diffusion MRI data:


In [1]:
#load general modules
import os
import numpy as np
import nibabel as nib

#load dipy modules
from dipy.core.gradients import gradient_table
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti, save_nifti

#load modules for visualization
import matplotlib.pyplot as plt

#define the paths to the data
scripts_dir = os.getcwd()
bids_dir = "../data/bids"
out_dir = "../data/derivatives"

#subject code to be used in this example
sub='01'

#load the pre-processed data
data_preproc, affine, data_preproc_img = load_nifti(f"{out_dir}/preprocessing/sub-{sub}/eddy_unwarped_images.nii.gz", return_img=True)

#load the bvals and bvecs
bvals, bvecs = read_bvals_bvecs(f"{bids_dir}/sub-{sub}/dwi/sub-{sub}_acq-AP_dwi.bval", f"{out_dir}/preprocessing/sub-{sub}/eddy_unwarped_images.eddy_rotated_bvecs")

#select data for DTI model fitting
#create a mask bo b-values less than 1300 s/mm2
bval_mask = bvals < 1300

#select the data for DTI model fitting
data_for_dti = data_preproc[..., bval_mask]

bvals_for_dti = bvals[bval_mask]
bvecs_for_dti = bvecs[bval_mask, :]

#load the gradient table
gtab_for_dti = gradient_table(bvals_for_dti, bvecs_for_dti)


#load the binary brain mask
brain_mask, affine_mask = load_nifti(f"{out_dir}/preprocessing/sub-{sub}/hifi_nodif_brain_mask.nii.gz")
masked_data = data_for_dti * brain_mask[..., np.newaxis]

### Step 1: Fit the DTI model to the data, and define a white matter mask by thresholding the FA map

Here we simply follow the same steps already covere in Tutorial 3 (Model Fitting).

In [2]:
#import the TensorModel from dipy
from dipy.reconst.dti import TensorModel

#fit the DTI model
dtimodel = TensorModel(gtab_for_dti)
dtifit = dtimodel.fit(data_for_dti, mask=brain_mask) 

#create a white matter mask by thresholding the FA map
fa = dtifit.fa
fa_mask = fa > 0.2


### Step 2: Estimate the principal direction of diffusion from the DTI model

For this we will use dipy's function peaks_from_model. This function provides a general strategy to estimate diffusion directions and it can be applied to different diffusion MRI reconstruction models. The function peaks_from_model requires a variable containing discrete directions for evaluation - this and other input parameters will be explained in more detail later on. For this example we simply load a default set of discrete directions available in dipy. 

In [3]:
from dipy.direction import peaks_from_model
from dipy.data import default_sphere

dti_peaks = peaks_from_model(dtimodel, masked_data, default_sphere,
                             relative_peak_threshold=.8,
                             min_separation_angle=45,
                             mask=fa_mask)

For quality control, we can visualise the estimated direction field using the fury python package:

In [4]:
from dipy.viz import window, actor, has_fury

if has_fury:
    scene = window.Scene()
    scene.add(actor.peak_slicer(dti_peaks.peak_dirs,
                               #dti_peaks.peak_values,
                               colors=None))

    window.record(scene, out_path=f'{out_dir}/DTI/sub-{sub}/dti_direction_field.png', size=(900, 900))
    window.show(scene, size=(800, 800))

/cognestic/miniconda3/envs/mri/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: We'll no longer accept the way you call the record function in future versions of FURY.

Here's how to call the Function record: record(scene='value', cam_pos='value', cam_focal='value', cam_view='value', out_path='value', path_numbering='value', n_frames='value', az_ang='value', magnification='value', size='value', reset_camera='value', screen_clip='value', stereo='value', verbose='value')

  exec(code_obj, self.user_global_ns, self.user_ns)


### Step 3: Define the seeds where streamline propagation will start

In Dipy, the seeds for tracking are established using the seeds_from_mask function from dipy.tracking.utils. In this example, a 2 × 2 × 2 grid of seeds is placed in each voxel within a sagittal slice of the corpus callosum, using a pre-defined mask. The voxels to be seeded are determined by the 'seed_mask' input, which here consists of the voxels labeled with the value 1 in the "CC_mask.nii.gz" file, indicating voxels in a sagittal slice of the corpus callosum. 

In [5]:
from dipy.tracking import utils

#load corpus callosum mask
cc_mask, affine_cc = load_nifti(f"{out_dir}/DTI/sub-{sub}/CC_mask.nii.gz")

seed_mask = (cc_mask == 1)
seeds = utils.seeds_from_mask(seed_mask, affine_cc, density=[2, 2, 2])

### Step 4: Defining stoping criteria


In this example, a streamline will continue to propgate until it reaches a voxel with FA lower than 0.2. We will use the dipy function ThresholdStoppingCriterion to define this stopping criterion area. You can find an extensive review of the various options for stopping criteria for tractography here:

https://workshop.dipy.org/documentation/1.7.0/examples_built/13_fiber_tracking/tracking_stopping_criterion/


In [6]:
from dipy.tracking.stopping_criterion import ThresholdStoppingCriterion

stopping_criterion = ThresholdStoppingCriterion(fa, 0.2)

### Step 5: Propagating the streamlines

We now have everything we need to start generating streamlines. In Dipy, this process requires initialization. To do this, we use the LocalTracking function from dipy.tracking.local_tracking. The streamlines themselves will then be generated using the Streamlines function from dipy.tracking.streamline:


In [7]:
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.streamline import Streamlines

#initialize the streamlines generator
streamlines_generator = LocalTracking(dti_peaks, stopping_criterion, seeds,
                                      affine=affine, step_size=0.5)

#generate the streamlines
streamlines = Streamlines(streamlines_generator)

These steps have created a deterministic set of streamlines from the principal diffusion direction estimated using the Diffusion Tensor model. 

Finally, we save the streamlines as a Trackvis file so it can be loaded into other software for visualization or further analysis.

In [8]:
from dipy.io.stateful_tractogram import Space, StatefulTractogram
from dipy.io.streamline import save_trk

sft = StatefulTractogram(streamlines, data_preproc_img, Space.RASMM)
save_trk(sft, f"{out_dir}/DTI/sub-{sub}/dti_CC_deterministic_tract.trk", streamlines)

The output can be visualised with the fury package:

In [9]:
from dipy.viz import colormap

if has_fury:
    # Prepare the display objects.
    color = colormap.line_colors(streamlines)

    streamlines_actor = actor.line(streamlines,
                                   colormap.line_colors(streamlines))

    # Create the 3D display.
    scene = window.Scene()
    scene.add(streamlines_actor)

    # Save still images for this static example. Or for interactivity use
    window.record(scene, out_path=f'{out_dir}/DTI/sub-{sub}/dti_tractogram_det.png', size=(800, 800))
    window.show(scene)

/cognestic/miniconda3/envs/mri/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: We'll no longer accept the way you call the line function in future versions of FURY.

Here's how to call the Function line: line(lines_value, colors='value', opacity='value', linewidth='value', spline_subdiv='value', lod='value', lod_points='value', lod_points_size='value', lookup_colormap='value', depth_cue='value', fake_tube='value')

  exec(code_obj, self.user_global_ns, self.user_ns)


# Exercises

1. In dipy, the total number of streamlines generated is determined by the number of seeds placed per voxel. This is controlled by the density parameter within utils.seeds_from_mask. In the example in this tutorial we used a 2x2x2 grid of seeds, i.e., 8 seeds per voxel. Try different seed densities and evaluate the impact of this parameter on the resulting tractogram.  

2. Another important parameter is the the FA treshold we set as the stopping criteria. Try different thresholds and evaluate the impact of this parameter on the resulting tractogram.  